In [ ]:
import pandas as pd
import numpy as np
import re

In [1]:
# defining input and output files 
input="credit_txn_v5.xlsx"
output="output_file.xlsx"


In [ ]:
df=pd.read_excel(input)

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])


In [ ]:
def date_periodicity_score(df, idx, lookback=6):
    row = df.loc[idx]
    narr = normalize(row["Narration"])

    history = df[
        (df.index != idx) &
        (df["Narration"].str.lower().str.contains(narr[:10], regex=False))
    ]

    if len(history) < 3:
        return 0

    gaps = history["Date"].sort_values().diff().dt.days.dropna()

    if gaps.between(25, 35).mean() > 0.6:
        return 80
    elif gaps.between(85, 95).mean() > 0.6:
        return 50
    else:
        return 0


In [ ]:
def amount_periodicity_score(df, idx):
    row = df.loc[idx]
    amt = row["Amount"]
    narr = normalize(row["Narration"])

    history = df[
        (df.index != idx) &
        (df["Narration"].str.lower().str.contains(narr[:10], regex=False))
    ]

    if len(history) < 3:
        return 0

    match_ratio = (
        abs(history["Amount"] - amt) / amt < 0.05
    ).mean()

    if match_ratio > 0.7:
        return 80
    elif match_ratio > 0.4:
        return 40
    else:
        return 0


In [ ]:
def confidence_score(df, idx):
    return (
        date_periodicity_score(df, idx)
        + amount_periodicity_score(df, idx)
    )


In [ ]:
df["score"] = [confidence_score(df, i) for i in df.index]